In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
# settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: C:\Users\klara\Universidade\Year 2\Scalable ML and DL\Lab1\mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
#from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url="https://api.waqi.info/feed/A98194/"
if not aqicn_url.startswith('https'):
    aqicn_url = " https://"+ aqicn_url
print(aqicn_url)
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-14 14:12:36,373 INFO: Initializing external client
2025-11-14 14:12:36,374 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 14:12:38,386 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279151
https://api.waqi.info/feed/A98194/


'{"country": "italy", "city": "rome", "street": "via-clelia", "aqicn_url": "https://api.waqi.info/feed/A98194/", "latitude": 41.89, "longitude": 12.48}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_lagged',
    version=4,
)
weather_fg = fs.get_feature_group(
    name='weather_more_features',
    version=2,
)
weather_fg.schema

[Feature('date', 'timestamp', 'Date of the daily weather measurement', False, False, False, None, None, 1716559),
 Feature('temperature_2m_mean', 'float', 'Mean temperature at 2 meters above ground (°C)', False, False, False, None, None, 1716559),
 Feature('apparent_temperature_mean', 'float', 'Maximum perceived (feels-like) temperature (°C)', False, False, False, None, None, 1716559),
 Feature('dewpoint_2m_mean', 'float', 'Mean dew point temperature at 2 meters above ground (°C)', False, False, False, None, None, 1716559),
 Feature('precipitation_sum', 'float', 'Total daily precipitation including rain and snow (mm)', False, False, False, None, None, 1716559),
 Feature('rain_sum', 'float', 'Total daily rainfall (mm)', False, False, False, None, None, 1716559),
 Feature('snowfall_sum', 'float', 'Total daily snowfall (cm)', False, False, False, None, None, 1716559),
 Feature('wind_speed_10m_max', 'float', 'Maximum wind speed at 10 meters above ground (m/s)', False, False, False, None, N

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import pandas as pd
from datetime import timedelta

# Get today's air quality data
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)

# Load full feature group as pandas DataFrame
air_quality_df = air_quality_fg.read()

# Convert 'today' and lags to date objects (drop any time component)
today_date = pd.to_datetime(today).date()
yesterday = today_date - timedelta(days=1)
before_yesterday = today_date - timedelta(days=2)
before_2_yesterday = today_date - timedelta(days=3)

# Helper: compare only on date part (no new columns)
def match_by_date(df, target_date):
    mask = df['date'].dt.date == target_date
    return df.loc[mask, 'pm25']

# Extract lag values
lag_1_series = match_by_date(air_quality_df, yesterday)
lag_2_series = match_by_date(air_quality_df, before_yesterday)
lag_3_series = match_by_date(air_quality_df, before_2_yesterday)

# Get scalar values (default to NaN if missing)
lag_1 = lag_1_series.iloc[0] if not lag_1_series.empty else float('nan')
lag_2 = lag_2_series.iloc[0] if not lag_2_series.empty else float('nan')
lag_3 = lag_3_series.iloc[0] if not lag_3_series.empty else float('nan')

# Add lag values as new columns to today's DF
aq_today_df['lag_1'] = lag_1
aq_today_df['lag_2'] = lag_2
aq_today_df['lag_3'] = lag_3

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.14s) 


,pm25,country,city,street,date,url,lag_1,lag_2,lag_3
0,23.0,italy,rome,via-clelia,2025-11-14,https://api.waqi.info/feed/A98194/,9.02,8.88,12.63


In [6]:
# PM2.5 for today
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
 6   lag_1    1 non-null      float32       
 7   lag_2    1 non-null      float32       
 8   lag_3    1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

,date,temperature_2m_mean,apparent_temperature_mean,dewpoint_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,city
0,2025-11-14,18.049999,18.275705,12.275568,0.0,0.0,0.0,3.319036,102.528801,417.0,0.233768,rome
1,2025-11-15,21.400000,20.399260,13.315816,0.0,0.0,0.0,14.113653,174.144073,408.0,0.272626,rome
2,2025-11-16,21.549999,20.658962,13.198903,0.0,0.0,0.0,13.084402,172.092926,429.0,0.282066,rome
3,2025-11-17,19.350000,17.602999,12.837326,0.2,0.2,0.0,18.118410,220.971649,387.0,0.240180,rome
4,2025-11-18,18.350000,18.174141,13.213208,0.1,0.1,0.0,8.225035,203.198608,405.0,0.226560,rome
5,2025-11-19,14.650000,14.139595,11.236645,0.1,0.1,0.0,6.120000,180.000000,318.0,0.163326,rome
6,2025-11-20,16.549999,13.849670,6.372600,0.0,0.0,0.0,12.240000,208.072388,320.0,0.213262,rome


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   apparent_temperature_mean    7 non-null      float32       
 3   dewpoint_2m_mean             7 non-null      float32       
 4   precipitation_sum            7 non-null      float32       
 5   rain_sum                     7 non-null      float32       
 6   snowfall_sum                 7 non-null      float32       
 7   wind_speed_10m_max           7 non-null      float32       
 8   wind_direction_10m_dominant  7 non-null      float32       
 9   shortwave_radiation_sum      7 non-null      float32       
 10  et0_fao_evapotranspiration   7 non-null      float32       
 11  city                         7 non-null      obje

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-14 14:12:46,231 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1703443


Uploading Dataframe: 100.00% |██████████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_lagged_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/air_quality_lagged_4_offline_fg_materialization/executions


(Job('air_quality_lagged_4_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747684
         }
       },
       "result": {
         "observed_value": 23.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T01:12:46.000230Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_e

In [11]:
# Insert new data
print("Columns in DataFrame to insert:")
print(daily_df.columns.tolist())
print("Feature group schema:")
print([f.name for f in weather_fg.features])

weather_fg.insert(daily_df, wait=True)

Columns in DataFrame to insert:
['date', 'temperature_2m_mean', 'apparent_temperature_mean', 'dewpoint_2m_mean', 'precipitation_sum', 'rain_sum', 'snowfall_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration', 'city']
Feature group schema:
['date', 'temperature_2m_mean', 'apparent_temperature_mean', 'dewpoint_2m_mean', 'precipitation_sum', 'rain_sum', 'snowfall_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration', 'city']
2025-11-14 14:15:07,597 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279151/fs/1265760/fg/1716559


Uploading Dataframe: 100.00% |██████████████████████████████████████████████████████████████████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_more_features_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279151/jobs/named/weather_more_features_2_offline_fg_materialization/executions
2025-11-14 14:15:23,310 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 14:15:26,506 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 14:16:52,408 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-14 14:16:58,884 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 14:16:59,066 INFO: Waiting for log aggregation to finish.
2025-11-14 14:17:07,724 INFO: Execution finished successfully.


(Job('weather_more_features_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 756737
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T01:15:07.000597Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
